In [1]:
%reset -f

In [2]:
import gc
gc.collect()

0

In [3]:
import psutil
psutil.virtual_memory()

svmem(total=64383717376, available=62788329472, percent=2.5, used=882528256, free=45877538816, active=1761943552, inactive=15867224064, buffers=326373376, cached=17297276928, shared=73728, slab=656019456)

## Import Modules

In [4]:
import pandas as pd
import numpy as np
import os
import math
# plot module
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm
# import lightgbm as lgb
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

## Read data

In [5]:
df = pd.read_csv('all.csv') # full dataset

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Cleaning

### Drop unnecessary variables 

In [6]:
df = df.drop(['DocumentNumberFormatted', 'ATTOM_ID', 'TransID', 'SitusCounty', 'PropertyAddressZIP',	'PropertyAddressZIP4', 'E_ZIP_CODE', 'ZIP', 'ZIP4', 'dr_ForeclosureRecordingDate','TaxAssessedValueImprovements','TaxAssessedValueLand','PreviousAssessedValue',
             'TaxMarketValueImprovements','TaxMarketValueLand','TaxMarketImprovementsPerc','AssessorPriorSaleAmount', 'AssessorLastSaleDate'], axis = 1)


### Drop Nan rows in "TaxAssessedImprovementsPerc"

In [7]:
df = df.dropna(subset=['TaxAssessedImprovementsPerc'])



### 0 to null

In [8]:
nan_list = ['Mortgage1Amount', 'TaxAssessedValueTotal', 'TaxMarketValueTotal', 'RoomsCount']
for col in nan_list:
  df[col] = df[col].replace(0, np.NaN)
  df[col] = df[col].replace(0.0, np.NaN)

In [9]:
e_var = ['E_DENSITY',	'E_POP10',	'E_HH10',	'E_FAMMAR18',	'E_NONFAMHH',	'E_TRAVHOME',	'E_OOCCHH',	'E_ROCCHH',	'E_MORTANY',	'E_NOMORT',	'E_MED_BLT',	'E_POV_TOTAL',	'E_MEDAGE',	'E_MEDHHSIZE',	'E_MEDRENT',	'E_MEDVALOCC',	'E_MEDHHINC','E_SQMILES']
for i in e_var:
  df.loc[df['E_SQMILES']==0, i] = np.NaN

### Checking the portion of forclosure 0 before dropna & choose the percentage for dropping

In [10]:
#checking missing%
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['%'] = round(missing/len(df),2)
missing = missing[missing['count'] != 0]

In [11]:
# ogirinal forclosure portion
df['dr_Foreclosure_Ind'].value_counts()[1]/len(df)

0.06388982076851155

In [12]:
# forclosure portion after dropping null (in variables having null below 0.7; I tired various values and 0.7 is the best)
missing_drop = missing[missing['%'] < 0.07].index.tolist()
ndf = df.dropna(subset = missing_drop)
ndf['dr_Foreclosure_Ind'].value_counts()[1]/len(ndf)

0.0676575761852413

### drop na about some columns

In [13]:
# dropping null (in variables having null below 0.7)
df = df.dropna(subset = missing_drop)

In [14]:
#checking missing% again
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['%'] = round(missing/len(df),2)
missing = missing[missing['count'] != 0]

In [15]:
# Final missing value list
df[missing.index].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6462144 entries, 0 to 7224133
Data columns (total 9 columns):
Mortgage1Amount            float64
StatusOwnerOccupiedFlag    float64
AreaBuilding               float64
RoomsCount                 float64
dr_DepthToWidth            float64
dr_YearBuilt               float64
dr_Ins_Date                object
dr_Ownership_Period        float64
dr_House_Age               float64
dtypes: float64(8), object(1)
memory usage: 493.0+ MB


### Creating new columns for missing values (comebine this later)

In [16]:
# get column names having nan
nan_check = df.columns[df.isna().any()].tolist()

# make new columns to check nan
mdf = pd.DataFrame()
for i in nan_check:
  v_name = i + '_Null'
  mdf[v_name] = df[i].isnull().apply(lambda x:  1 if x == True else 0)

### Convert date to unixtime

In [17]:
def time_f(x):
    try:
        return time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple())
    except:
        return np.NaN

In [18]:
import time
import datetime


df['dr_Ins_Date'] = df['dr_Ins_Date'].astype(str)
df['dr_Ins_Date'] = df['dr_Ins_Date'].apply(lambda x: time_f(x))

In [19]:
df['dr_Ins_Date'].head()

0    1.197590e+09
1    1.286323e+09
2    1.355443e+09
3    1.349136e+09
4             NaN
Name: dr_Ins_Date, dtype: float64

### One-hot-encoding

In [20]:
df = pd.get_dummies(df, columns = ['dr_Value_Change'], prefix='dr_Value_Change')

### Combine missing check

In [21]:
df = pd.concat([df, mdf], axis=1)

### Impute Missing Value

In [22]:
#checking missing% again
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['%'] = round(missing/len(df),2)
missing = missing[missing['count'] != 0]

In [23]:
# median
for i in missing.index:
    df[i] = df[i].fillna(value=df[i].median())

## Standard Scaling

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
df_sc = scaler.transform(df)
df_sc = pd.DataFrame(df_sc, columns=df.columns)

In [25]:
df_sc.head()

,Mortgage1Amount,StatusOwnerOccupiedFlag,TaxYearAssessed,TaxAssessedValueTotal,TaxAssessedImprovementsPerc,TaxMarketValueTotal,AssessorLastSaleAmount,AreaBuilding,RoomsCount,E_SQMILES,...,dr_Value_Change_NC,Mortgage1Amount_Null,StatusOwnerOccupiedFlag_Null,AreaBuilding_Null,RoomsCount_Null,dr_DepthToWidth_Null,dr_YearBuilt_Null,dr_Ins_Date_Null,dr_Ownership_Period_Null,dr_House_Age_Null
0,0.000054,1.0,1.0,0.000317,0.008070,0.000317,0.000000,0.000074,0.000603,0.000373,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,0.000005,1.0,1.0,0.000247,0.008525,0.000247,0.000476,0.000094,0.000603,0.000373,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.000144,1.0,1.0,0.000288,0.009548,0.000288,0.000535,0.000076,0.000603,0.000373,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.000074,1.0,1.0,0.000194,0.009093,0.000194,0.000000,0.000062,0.000603,0.000373,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.000198,1.0,1.0,0.000207,0.007843,0.000207,0.000318,0.000065,0.000603,0.000373,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


## SMOTE

In [26]:
from sklearn.model_selection import train_test_split
X = df_sc.drop('dr_Foreclosure_Ind', axis=1)
y = df_sc['dr_Foreclosure_Ind']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [27]:
y_test.value_counts()

0.0    2409514
1.0     175344
Name: dr_Foreclosure_Ind, dtype: int64

In [28]:
# !pip install -U imbalanced-learn

In [29]:
from imblearn.over_sampling import SMOTE
smote = SMOTE( random_state=0)

X_train, y_train = smote.fit_resample(X_train, y_train)

## Fast.AI

In [30]:
# on Terminal
# conda uninstall --force jpeg libtiff -y
# conda install -c conda-forge libjpeg-turbo pillow==6.0.0
# CC="cc -mavx2" pip install --no-cache-dir -U --force-reinstall --no-binary :all: --compile pillow-simd

In [31]:
# !pip install git+https://github.com/fastai/fastai.git

In [32]:
from fastai.tabular import *

In [33]:
procs = [FillMissing, Normalize]
valid_idx = range(len(df)-30000, len(df))
dep_var = 'dr_Foreclosure_Ind'
con = df.drop('dr_Foreclosure_Ind', axis=1).columns.tolist()

In [34]:
XX = pd.DataFrame(X_train, columns=df.drop('dr_Foreclosure_Ind', axis=1).columns)
yy = pd.DataFrame(y_train, columns=['dr_Foreclosure_Ind'])
ndf = pd.concat([XX, yy], axis=1)
test_df = pd.DataFrame(X_test, columns=df.drop('dr_Foreclosure_Ind', axis=1).columns)

In [35]:
path = '/home/ec2-user/SageMaker'

In [36]:
data = TabularDataBunch.from_df(path=path, df=ndf, 
                                dep_var=dep_var, 
                                valid_idx=valid_idx, 
                                cat_names=[], 
                                cont_names=con, 
#                                 bs=2048,
                                procs=[],
                                test_df=test_df)

In [37]:
learn = tabular_learner(data, layers=[200,100], wd=0.01, y_range=[0,1])

In [38]:
# learn.lr_find()
# learn.recorder.plot()

In [39]:
learn.fit_one_cycle(7, 1.00E-03)

epoch,train_loss,valid_loss,time
0,0.142585,0.116105,13:36
1,0.132706,0.113120,13:36
2,0.132182,0.107504,13:36
3,0.129818,0.110171,13:39
4,0.126564,0.108488,13:39
5,0.116819,0.097840,13:39
6,0.120198,0.096070,13:40


In [40]:
# learn.unfreeze()

In [41]:
# learn.fit_one_cycle(7,  max_lr=1e-2)

In [42]:
test_preds, _ = learn.get_preds(ds_type=DatasetType.Test)
target_preds = to_np(test_preds)
target_preds = target_preds.reshape((target_preds.shape[0],))
target_preds = pd.Series(target_preds)

In [57]:
prediction = target_preds.apply(lambda x: 1 if x>=.85 else 0)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,prediction))

[[2347507   62007]
 [ 110368   64976]]


In [59]:
xx =confusion_matrix(y_test,prediction)[0][1] + confusion_matrix(y_test,prediction)[1][0]
yy = xx + confusion_matrix(y_test,prediction)[0][0] + confusion_matrix(y_test,prediction)[1][1]
round(1- (xx / yy),4)

0.9333

In [58]:
print(classification_report(prediction, y_test))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96   2457875
           1       0.37      0.51      0.43    126983

    accuracy                           0.93   2584858
   macro avg       0.67      0.73      0.70   2584858
weighted avg       0.94      0.93      0.94   2584858



## Logistic

In [60]:
import statsmodels.api as sm

In [66]:
logit = sm.Logit(y_train, X_train)

In [ ]:
result = logit.fit(maxiter=10000)

In [ ]:
print result.summary()

In [ ]:
y_pred_logit = logit.predict(X_test)

In [ ]:
print(classification_report(y_pred_logit, y_test))

In [ ]:
#Building predictive model based on all selected variables 
multi_model = sm.formula.glm("dr_Foreclosure_Ind~C(Mortgage1Amount)+C(AreaBuilding)+C(E_SQMILES)+C(E_DENSITY)\
                             +C(E_MEDAGE)+C(E_MEDHHSIZE)+C(dr_PERC_VACUNIT)+C(dr_MktValueToIncome)\
                             +C(dr_DepthToWidth)+C(dr_Annual_PopGrowth10)+C(TaxAssessedValueLand)+C(TaxMarketValueTotal)+C(TaxMarketValueLand)\
                             +C(E_POP10)+C(E_FAMMAR18)+C(E_TRAVHOME)+C(E_NONFAMHH)+C(E_OOCCHH)\
                             +C(E_ROCCHH)+C(E_MORTANY)+C(E_NOMORT)+C(E_MEDHHINC)\
                             +C(dr_Condo_Ind)+C(dr_Ownership_Period)+C(dr_House_Age)+C(dr_Value_Change)\
                             +C(X_PERC_CDPD60)+C(X_Bankruptcy_Index)+C(X_Risk_Score)+C(X_Vantage_Score)\
                             +C(dr_TRAVLTIME)"\
                             , data=X_train, family=sm.families.Binomial()).fit()
print(multi_model.summary())

In [ ]:
X_test = X_test.drop('dr_Foreclosure_Ind', axis=1)

In [ ]:
pred = multi_model.predict(X_test)
pred.mean()

In [ ]:
prediction = pred.apply(lambda x: 1 if x>=.5 else 0)

In [ ]:
prediction.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(prediction, y_test))

In [ ]:
#accuracy
(141+5187)/(141+5187+71680+164)

## Deep Learning

In [ ]:
import tensorflow as tf
feat_cols = []
for i in df_feat.columns:
  feat_cols += [tf.feature_column.numeric_column(i)]

In [ ]:
X_train = pd.DataFrame(X_train, columns=X.columns)
y_train = pd.Series(y_train)

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=128, num_epochs=30 ,shuffle=True)

In [ ]:
classifier = tf.estimator.DNNClassifier(hidden_units=[50, 150, 50], n_classes=2,feature_columns=feat_cols) #, optimizer=tf.train.AdamOptimizer(1e-2))
classifier.train(input_fn=input_func,steps=5000)

In [ ]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)
note_predictions = list(classifier.predict(input_fn=pred_fn))
note_predictions[0]
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [ ]:
print(confusion_matrix(y_test,final_preds))

In [ ]:
print(classification_report(y_test,final_preds))

## LGMB

In [ ]:
# from sklearn.model_selection import train_test_split
# X = df.drop('dr_Foreclosure_Ind', axis=1)
# y = df['dr_Foreclosure_Ind']
# # X = X.astype('category')
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

# lgb_train = lgb.Dataset(X_train, label = y_train, feature_name = col_cat, categorical_feature = col_cat)
# lgb_test = lgb.Dataset(X_test, label = y_test, feature_name = col_cat, categorical_feature = col_cat)

In [ ]:
# LGBM 1
params = {
    'task': 'train',
    'objective': 'binary',
    'metric': 'binary_error',
    'verbose': 50
}

In [ ]:
lgbm = lgb.train(params, 
                 lgb_train, 
                 num_boost_round=500,
                 early_stopping_rounds = 100,
                 valid_sets=[lgb_train, lgb_test],
                 valid_names=['train', 'test'],
                 )

In [ ]:
pred = lgbm.predict(X_test)
pred = pd.Series(pred)
pred = pred.apply(lambda x: 1 if x>=.5 else 0)
print(classification_report(pred,y_test))

In [ ]:
print(confusion_matrix(pred, y_test))

In [ ]:
(895+3406)/(895+3406+71249+1608)

### Feature importance

In [ ]:
!pip install shap
import shap

In [ ]:
%time shap_values = shap.TreeExplainer(lgbm).shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.dependence_plot("Mortgage1Amount", shap_values, X_test)

In [ ]:
fi =pd.DataFrame(lgbm.feature_importance(), index=X.columns)

In [ ]:
fi.sort_values(by=[0], ascending=False)

In [ ]:
sns.barplot(x="Value", y="Feature", data=lgbm.feature_importance

In [ ]:
# LGBM2

clf = lgb.LGBMClassifier(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.01)

In [ ]:
fit_params={"early_stopping_rounds":1000, 
            "eval_metric" : 'binary_error', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': 'auto' # that's actually the default
           }

In [ ]:
clf.fit(X_train, y_train, **fit_params)

In [ ]:
pred = clf.predict(X_test)
print(classification_report(pred,y_test))

In [ ]:
print(confusion_matrix(pred, y_test))

In [ ]:
(166+3871)/(166+3871+62269+1054)

In [ ]:
# LGBM 3

fit_params = {
    "objective": "binary",
    "metric": "binary_error",
    "boosting": 'gbdt',
    "max_depth": -1,
    "num_leaves": 13,
    "learning_rate": 0.01,
    "bagging_freq": 5,
    "bagging_fraction": 0.4,
    "feature_fraction": 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    # "lambda_l1" : 5,
    # "lambda_l2" : 5,
    "bagging_seed": 314,
    "verbosity": 1,
    "seed": 314
}

In [ ]:
evals_result = {}
lgb_clf = lgb.train(fit_params,
                            lgb_train,
                            100000,
                            valid_sets=[lgb_train, lgb_test],
                            early_stopping_rounds=3000,
                            verbose_eval=1000,
                            evals_result=evals_result
                            )

In [ ]:
pred = lgb_clf.predict(X_test)
pred = pd.Series(pred)
pred = pred.apply(lambda x: 1 if x>=.5 else 0)
print(classification_report(pred,y_test))


In [ ]:
print(confusion_matrix(pred, y_test))

In [ ]:
(174+4958)/(174+4958+342+71715)

## LGBM - No grouping

## **REGROUING** 


In [ ]:
# New DataFrame for groupnig values
gdf = pd.DataFrame()
gdf['dr_Foreclosure_Ind'] = df['dr_Foreclosure_Ind']

In [ ]:
# Manual grouping function
def m_grp(col, bins):
  bin = bins + [df[col].max()]
  labels = bin[:-1]
  gdf[col] = pd.cut(df[col], bin, labels = labels, include_lowest=True).astype(str)
  
# Natural grouping function
def n_grp(col, min, max, interval):
  bin = list(range(min, max, interval)) + [df[col].max()]
  labels = bin[:-1]
#   labels = list(range(len(bin)-1))
  gdf[col] = pd.cut(df[col], bin, labels=labels, include_lowest=True).astype(str)
  
#Equal grouping
# function
def e_grp(col, bins):
  spc = np.linspace(df[col].min(), df[col].max(), bins).tolist()
  bin = [round(x,2) for x in spc]
  labels = bin[:-1]
  gdf[col] = pd.cut(df[col], bin, labels=labels, include_lowest=True).astype(str)

In [ ]:
#1.Mortgage1Amount
m_grp('Mortgage1Amount', [0, 50000, 100000, 200000, 300000, 400000, 500000])
gdf.loc[gdf['Mortgage1Amount'] == 'nan', 'Mortgage1Amount'] = 'NA'
gdf.loc[gdf['Mortgage1Amount'] == '0.0', 'Mortgage1Amount'] = 'NA'
gdf.loc[gdf['Mortgage1Amount'] == '50000.0', 'Mortgage1Amount'] = 'LT50K'
gdf.loc[gdf['Mortgage1Amount'] == '100000.0', 'Mortgage1Amount'] = '100-200K'
gdf.loc[gdf['Mortgage1Amount'] == '200000.0', 'Mortgage1Amount'] = '200K-300K'
gdf.loc[gdf['Mortgage1Amount'] == '300000.0', 'Mortgage1Amount'] = '300K-400K'
gdf.loc[gdf['Mortgage1Amount'] == '400000.0', 'Mortgage1Amount'] = '400K-500K'
gdf.loc[gdf['Mortgage1Amount'] == '500000.0', 'Mortgage1Amount'] = 'GT500K'

#2. 'AreaBuilding'
m_grp('AreaBuilding', [0, 500, 1000, 1500, 2000, 3000])
gdf.loc[gdf['AreaBuilding'] == 'nan', 'AreaBuilding'] = 'NA'
gdf.loc[gdf['AreaBuilding'] == '0.0', 'AreaBuilding'] = 'NA'
gdf.loc[gdf['AreaBuilding'] == '500.0', 'AreaBuilding'] = 'LT500SQFT'
gdf.loc[gdf['AreaBuilding'] == '1000.0', 'AreaBuilding'] = '500-1000SQFT'
gdf.loc[gdf['AreaBuilding'] == '1500.0', 'AreaBuilding'] = '1000-1500SQFT'
gdf.loc[gdf['AreaBuilding'] == '2000.0', 'AreaBuilding'] = '1500-2000SQFT'
gdf.loc[gdf['AreaBuilding'] == '3000.0', 'AreaBuilding'] = 'LT3000SQFT'

#3.RoomsCount
m_grp('RoomsCount', [0, 5, 6, 8, 11, 12])
gdf.loc[gdf['RoomsCount'] == 'nan', 'RoomsCount'] = 'NA'
gdf.loc[gdf['RoomsCount'] == '0.0', 'RoomsCount'] = 'NA'
gdf.loc[gdf['RoomsCount'] == '5.0', 'RoomsCount'] = 'LT5'
gdf.loc[gdf['RoomsCount'] == '6.0', 'RoomsCount'] = '5-6'
gdf.loc[gdf['RoomsCount'] == '8.0', 'RoomsCount'] = '7-8'
gdf.loc[gdf['RoomsCount'] == '11.0', 'RoomsCount'] = '8-11'
gdf.loc[gdf['RoomsCount'] == '12.0', 'RoomsCount'] = 'GT12'

#4. E_SQMILES
m_grp('E_SQMILES', [0, 2, 5, 10, 20, 50]) #median 9.27

gdf.loc[gdf['E_SQMILES'] == 'nan', 'E_SQMILES'] = 'NA'
gdf.loc[gdf['E_SQMILES'] == '0.0', 'E_SQMILES'] = 'NA'
gdf.loc[gdf['E_SQMILES'] == '2.0', 'E_SQMILES'] = 'LT500SQML'
gdf.loc[gdf['E_SQMILES'] == '5.0', 'E_SQMILES'] = '500-1000SQML'
gdf.loc[gdf['E_SQMILES'] == '10.0','E_SQMILES'] = '1000-1500SQML'
gdf.loc[gdf['E_SQMILES'] == '20.0','E_SQMILES'] = '1500-2000SQML'
gdf.loc[gdf['E_SQMILES'] == '50.0','E_SQMILES'] = 'LT3000SQML'

#5.E_DENSITY
m_grp('E_DENSITY', [0, 100, 500, 1000, 2500, 5000, 10000, 40000]) #med 2647.1
gdf.loc[gdf['E_DENSITY'] == 'nan', 'E_DENSITY'] = 'NA'
gdf.loc[gdf['E_DENSITY'] == '0.0', 'E_DENSITY'] = 'NA'

#6. E_MED_BLT
m_grp('E_MED_BLT', [0, 1900, 1910, 1920, 1930, 1940, 1950, 1960 , 1970, 1980]) #med 1955.0
#gdf.loc[gdf['E_MED_BLT'] == '0', 'E_MED_BLT'] = 'NA'


#7.'E_MEDAGE'
m_grp('E_MEDAGE', [0, 25, 35, 45, 55]) #gen Z, millenial, gen x, boomer
#gdf.loc[gdf['E_MEDAGE'] == 'nan', 'E_MEDAGE'] = 'NA'

#8.'E_MEDHHSIZE'
m_grp('E_MEDHHSIZE', [0, 1, 1.5, 2, 2.5, 3, 3.5, 4]) #median 2.5
gdf.loc[gdf['E_MEDHHSIZE'] == 'nan', 'E_MEDHHSIZE'] = 'NA'
gdf.loc[gdf['E_MEDHHSIZE'] == '0.0', 'E_MEDHHSIZE'] = 'NA'


#9. dr_PERC_VACUNIT'
m_grp('dr_PERC_VACUNIT', [0, 2, 4, 6, 8, 10, 12, 14, 16]) #median 7.07

gdf.loc[gdf['dr_PERC_VACUNIT'] == 'nan', 'dr_PERC_VACUNIT'] = 'NA'
gdf.loc[gdf['dr_PERC_VACUNIT'] == '0.0', 'dr_PERC_VACUNIT'] = 'NA'


#10. dr_MktValueToIncome
# Rebin
m_grp('dr_MktValueToIncome', [0, .5, 1, 2, 4, 6, 8, 10, 20]) #median 3.05

gdf.loc[gdf['dr_MktValueToIncome'] == 'nan', 'dr_MktValueToIncome'] = 'NA'
gdf.loc[gdf['dr_MktValueToIncome'] == '0.0', 'dr_MktValueToIncome'] = 'NA'

#11. dr_DepthToWidth'
m_grp('dr_DepthToWidth', [0, 1, 2, 3, 4, 5, 10, 20]) #med 2.3

gdf.loc[gdf['dr_DepthToWidth'] == 'nan', 'dr_DepthToWidth'] = 'NA'
gdf.loc[gdf['dr_DepthToWidth'] == '0.0', 'dr_DepthToWidth'] = 'NA'
"""
gdf.loc[gdf['AreaBuilding'] == '500.0', 'AreaBuilding'] = 'LT500SQFT'
gdf.loc[gdf['AreaBuilding'] == '1000.0', 'AreaBuilding'] = '500-1000SQFT'
gdf.loc[gdf['AreaBuilding'] == '1500.0', 'AreaBuilding'] = '1000-1500SQFT'
gdf.loc[gdf['AreaBuilding'] == '2000.0', 'AreaBuilding'] = '1500-2000SQFT'
gdf.loc[gdf['AreaBuilding'] == '3000.0', 'AreaBuilding'] = 'LT3000SQFT'
"""

#12. 'dr_Annual_PopGrowth10
m_grp('dr_Annual_PopGrowth10', [-4,-2,-1,-0.5,0,0.5,1,2,4])
gdf.loc[gdf['dr_Annual_PopGrowth10'] == 'nan', 'dr_Annual_PopGrowth10'] = 'NA'


#13. 'StatusOwnerOccupiedFlag
gdf['StatusOwnerOccupiedFlag'] = df['StatusOwnerOccupiedFlag'].astype(str)
gdf.loc[gdf['StatusOwnerOccupiedFlag'] == 'nan', 'StatusOwnerOccupiedFlag'] = 'NA'

# Alternative
#14. 'TaxAssessedValueTotal
n_grp('TaxAssessedValueTotal',0,250000,25000)
# m_grp('TaxAssessedValueTotal', [0,25000,50000,75000,125000,150000,175000,200000,225000,250000]) #100000 is missing
gdf.loc[gdf['TaxAssessedValueTotal'] == 'nan', 'TaxAssessedValueTotal'] = 'NA'
gdf.loc[gdf['TaxAssessedValueTotal'] == '0.0', 'TaxAssessedValueTotal'] = 'NA'


# Alternative
#15. 'TaxAssessedValueImprovements
n_grp('TaxAssessedValueImprovements', min = 0, max = 200000, interval=20000)
# m_grp('TaxAssessedValueImprovements', [0,20000,40000,60000,80000,100000,120000,140000,160000,180000,200000])

gdf.loc[gdf['TaxAssessedValueImprovements'] == 'nan', 'TaxAssessedValueImprovements'] = 'NA'
gdf.loc[gdf['TaxAssessedValueImprovements'] == '0.0', 'TaxAssessedValueImprovements'] = 'NA'

#16. 'TaxAssessedValueLand
# n_grp('TaxAssessedValueLand', min = 0, max = 59000, interval=5900)

m_grp('TaxAssessedValueLand', [0,5900,11800,17700,23600,29500,35400,41300,47200,53100,59000])

gdf.loc[gdf['TaxAssessedValueLand'] == 'nan', 'TaxAssessedValueLand'] = 'NA'
gdf.loc[gdf['TaxAssessedValueLand'] == '0.0', 'TaxAssessedValueLand'] = 'NA'

#17. 'TaxAssessedImprovementsPerc
m_grp('TaxAssessedImprovementsPerc', [0.0,11.11,22.22,33.33,44.44,55.56,66.67,77.78,88.89])

gdf.loc[gdf['TaxAssessedImprovementsPerc'] == 'nan', 'TaxAssessedImprovementsPerc'] = 'NA'
gdf.loc[gdf['TaxAssessedImprovementsPerc'] == '0.0', 'TaxAssessedImprovementsPerc'] = 'NA'


#18. 'PreviousAssessedValue
m_grp('PreviousAssessedValue', [0,25000,50000,75000,100000,125000,150000,175000,200000,225000,250000])

gdf.loc[gdf['PreviousAssessedValue'] == 'nan', 'PreviousAssessedValue'] = 'NA'
gdf.loc[gdf['PreviousAssessedValue'] == '0.0', 'PreviousAssessedValue'] = 'NA'

#19. 'TaxMarketValueTotal
m_grp('TaxMarketValueTotal', [0,70000,140000,210000,280000,350000,420000,490000,560000,630000,700000])

gdf.loc[gdf['TaxMarketValueTotal'] == 'nan', 'TaxMarketValueTotal'] = 'NA'
gdf.loc[gdf['TaxMarketValueTotal'] == '0.0', 'TaxMarketValueTotal'] = 'NA'

#20. 'TaxMarketValueImprovements
m_grp('TaxMarketValueImprovements', [0, 42000, 84000, 126000, 168000, 210000, 252000, 294000, 336000, 378000,420000])

gdf.loc[gdf['TaxMarketValueImprovements'] == 'nan', 'TaxMarketValueImprovements'] = 'NA'
gdf.loc[gdf['TaxMarketValueImprovements'] == '0.0', 'TaxMarketValueImprovements'] = 'NA'

#21. 'TaxMarketValueLand
m_grp('TaxMarketValueLand', [0, 21000, 42000, 63000, 84000, 105000, 126000, 147000, 168000, 189000,210000])

gdf.loc[gdf['TaxMarketValueLand'] == 'nan', 'TaxMarketValueLand'] = 'NA'
gdf.loc[gdf['TaxMarketValueLand'] == '0.0', 'TaxMarketValueLand'] = 'NA'

#22. 'AssessorLastSaleAmount
m_grp('AssessorLastSaleAmount', [0, 21000, 42000, 63000, 84000, 105000, 126000, 147000, 168000, 189000,210000])

gdf.loc[gdf['AssessorLastSaleAmount'] == 'nan', 'AssessorLastSaleAmount'] = 'NA'
gdf.loc[gdf['AssessorLastSaleAmount'] == '0.0', 'AssessorLastSaleAmount'] = 'NA'

#23. 'E_POP10
#m_grp('E_POP10', [0.0,9781.33,19562.67,29344.0,39125.33,48906.67,58688.0,68469.33,78250.67,88032.0])
e_grp('E_POP10', 10)

gdf.loc[gdf['E_POP10'] == '0.0', 'E_POP10'] = 'NA'

#24. 'E_HH10
e_grp('E_HH10', 10)
gdf.loc[gdf['E_HH10'] == '0.0', 'E_HH10'] = 'NA'

#25. 'E_FAMMAR18
e_grp('E_FAMMAR18', 10)
gdf.loc[gdf['E_FAMMAR18'] == '0.0', 'E_FAMMAR18'] = 'NA'

#26. 'E_NONFAMHH - regrouped - from equal to manual
m_grp('E_NONFAMHH', [0.0,2000.00,4000.00,6000.00,8000.00,10000.00])
gdf.loc[gdf['E_NONFAMHH'] == '0.0', 'E_NONFAMHH'] = 'NA'

#27. 'E_TRAVHOME
e_grp('E_TRAVHOME', 10)
gdf.loc[gdf['E_TRAVHOME'] == '0.0', 'E_TRAVHOME'] = 'NA'

#28. 'E_OOCCHH
e_grp('E_OOCCHH', 10)
gdf.loc[gdf['E_OOCCHH'] == '0.0', 'E_OOCCHH'] = 'NA'

#29. 'E_ROCCHH
m_grp('E_ROCCHH', [0,2000,4000,6000,8000])
gdf.loc[gdf['E_ROCCHH'] == '0.0', 'E_ROCCHH'] = 'NA'

#30. E_MORTANY
m_grp('E_MORTANY', [0,2000,4000,6000,8000])
gdf.loc[gdf['E_MORTANY'] == '0.0', 'E_MORTANY'] = 'NA'

#31. E_NOMORT
m_grp('E_NOMORT', [0,1000,2000,3000,4000])

#33. E_MEDVALOCC
#e_grp('E_MEDVALOCC', 10)
#gdf.loc[gdf['E_MEDVALOCC'] == '0.0', 'E_MEDVALOCC'] = 'NA'

# 34.E_MEDHHINC
e_grp('E_MEDHHINC', 10)

# 35.dr_PERC_ASSO_ABOVE
e_grp('dr_PERC_ASSO_ABOVE', 10)
gdf.loc[gdf['dr_PERC_ASSO_ABOVE'] == 'nan', 'dr_PERC_ASSO_ABOVE'] = 'NA'
gdf.loc[gdf['dr_PERC_ASSO_ABOVE'] == '0.0', 'dr_PERC_ASSO_ABOVE'] = 'NA'

#38. dr_Condo_Ind
gdf['dr_Condo_Ind'] = df['dr_Condo_Ind'].astype(str)
gdf.loc[gdf['dr_Condo_Ind'] == 'nan', 'dr_Condo_Ind'] = 'NA'

#39. dr_Ownership_Period
n_grp('dr_Ownership_Period', min = 0, max = 16, interval=2)
gdf.loc[gdf['dr_Ownership_Period'] == 'nan', 'dr_Ownership_Period'] = 'NA'

#40. dr_House_Age
n_grp('dr_House_Age', min = 1, max = 120, interval=12)
gdf.loc[gdf['dr_House_Age'] == 'nan', 'dr_House_Age'] = 'NA'

#41. dr_Value_Change
gdf['dr_Value_Change'] = df['dr_Value_Change'].astype(str)
gdf.loc[gdf['dr_Value_Change'] == 'nan', 'dr_Value_Change'] = 'NA'


#43. X_PERC_CDPD60
e_grp('X_PERC_CDPD60',10)

gdf.loc[gdf['X_PERC_CDPD60'] == 'nan', 'X_PERC_CDPD60'] = 'NA'
gdf.loc[gdf['X_PERC_CDPD60'] == '0.0', 'X_PERC_CDPD60'] = 'NA'

#44. X_PERC_CForeclosure
m_grp('X_PERC_CForeclosure', [0,1,2,3,4,5])

gdf.loc[gdf['X_PERC_CForeclosure'] == 'nan', 'X_PERC_CForeclosure'] = 'NA'
gdf.loc[gdf['X_PERC_CForeclosure'] == '0.0', 'X_PERC_CForeclosure'] = 'NA'


#45. X_PERC_CDerogatory
e_grp('X_PERC_CDerogatory',10)

gdf.loc[gdf['X_PERC_CDerogatory'] == 'nan', 'X_PERC_CDerogatory'] = 'NA'
gdf.loc[gdf['X_PERC_CDerogatory'] == '0.0', 'X_PERC_CDerogatory'] = 'NA'

#46. X_PERC_CBankruptcy
m_grp('X_PERC_CBankruptcy', [0,1,2,3,4,5])

gdf.loc[gdf['X_PERC_CBankruptcy'] == 'nan', 'X_PERC_CBankruptcy'] = 'NA'
gdf.loc[gdf['X_PERC_CBankruptcy'] == '0.0', 'X_PERC_CBankruptcy'] = 'NA'

#47. X_PERC_MDPD60
m_grp('X_PERC_MDPD60', [0,1,2,3,4,5])

gdf.loc[gdf['X_PERC_MDPD60'] == 'nan', 'X_PERC_MDPD60'] = 'NA'
gdf.loc[gdf['X_PERC_MDPD60'] == '0.0', 'X_PERC_MDPD60'] = 'NA'

#48. X_PERC_MDerogatory
m_grp('X_PERC_MDerogatory', [0,1,2,3,4,5])

gdf.loc[gdf['X_PERC_MDerogatory'] == 'nan', 'X_PERC_MDerogatory'] = 'NA'
gdf.loc[gdf['X_PERC_MDerogatory'] == '0.0', 'X_PERC_MDerogatory'] = 'NA'

#49. X_PERC_MBankruptcy
m_grp('X_PERC_MBankruptcy', [0,1,2,3,4,5])

gdf.loc[gdf['X_PERC_MBankruptcy'] == 'nan', 'X_PERC_MBankruptcy'] = 'NA'
gdf.loc[gdf['X_PERC_MBankruptcy'] == '0.0', 'X_PERC_MBankruptcy'] = 'NA'

#50. X_Bankruptcy_Index
m_grp('X_Bankruptcy_Index', [0,270,275,280,285,290,295])

gdf.loc[gdf['X_Bankruptcy_Index'] == 'nan', 'X_Bankruptcy_Index'] = 'NA'
gdf.loc[gdf['X_Bankruptcy_Index'] == '0.0', 'X_Bankruptcy_Index'] = 'NA'

#51. X_Risk_Score
m_grp('X_Risk_Score', [0,500,600,700,800])
#e_grp('X_Risk_Score', 9)

gdf.loc[gdf['X_Risk_Score'] == 'nan', 'X_Risk_Score'] = 'NA'
gdf.loc[gdf['X_Risk_Score'] == '0.0', 'X_Risk_Score'] = 'NA'


#52. X_Vantage_Score
#e_grp('X_Vantage_Score', 9)
m_grp('X_Vantage_Score', [0,600,700,800,900])

gdf.loc[gdf['X_Vantage_Score'] == 'nan', 'X_Vantage_Score'] = 'NA'
gdf.loc[gdf['X_Vantage_Score'] == '0.0', 'X_Vantage_Score'] = 'NA'

#53. E_MEDRENT
m_grp('E_MEDRENT', [0,200,400,600,800])

gdf.loc[gdf['E_MEDRENT'] == 'nan', 'E_MEDRENT'] = 'NA'
gdf.loc[gdf['E_MEDRENT'] == '0', 'E_MEDRENT'] = 'NA'


#54. dr_TRAVLTIME
m_grp('dr_TRAVLTIME', [0.00,20.00,25.00,30.00,35.00,40.00,45.00])

gdf.loc[gdf['dr_TRAVLTIME'] == 'nan', 'dr_TRAVLTIME'] = 'NA'
gdf.loc[gdf['dr_TRAVLTIME'] == '0.0', 'dr_TRAVLTIME'] = 'NA'

In [ ]:
gdf.head()

In [ ]:
#del df due to memory issue
del df

In [ ]:
# from sklearn.model_selection import train_test_split
# X = df.drop('dr_Foreclosure_Ind', axis=1)
# y = df['dr_Foreclosure_Ind']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
from imblearn.over_sampling import SMOTENC
smote_nc = SMOTENC(categorical_features=[0,1], random_state=0)

X_train, y_train = smote_nc.fit_resample(X_train, y_train)

In [ ]:
col = gdf.columns.tolist()

In [ ]:
df = df[col]

In [ ]:
# Get the missing value for each independent variables from the dataset
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['percent']  = missing/len(df)
missing = missing.sort_values(['percent'], ascending=False)
missing.head()

In [ ]:
# Drop variables having missing value over 40%
# df = df.drop(['AssessorPriorSaleAmount', 'TaxMarketImprovementsPerc'], axis = 1)
# missing = missing.drop(['AssessorPriorSaleAmount', 'TaxMarketImprovementsPerc'])
# Arrange Missing value again
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['percent']  = missing/len(df)
missing = missing.sort_values(['percent'], ascending=False)

In [ ]:
# Get index if NA is not 0
missing_name = missing[missing['count'] != 0].index.values.tolist(); missing_name

# # Impute missing values
for i in missing_name:
    df[i] = df[i].fillna(value=df[i].median())

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('dr_Foreclosure_Ind', axis=1)
y = df['dr_Foreclosure_Ind']
X = X.astype('category')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

In [ ]:
# LGBM 1
params = {
    'task': 'train',
    'objective': 'binary',
    'metric': 'auc',
    'verbose' : 100
}

In [ ]:
lgbm = lgb.train(params, 
                 lgb_train, 
                 num_boost_round=500,
                 early_stopping_rounds = 100,
                 valid_sets=[lgb_train, lgb_test],
                 valid_names=['train', 'test']                 
                 )

In [ ]:
pred = lgbm.predict(X_test)
pred = pd.Series(pred)
pred = pred.apply(lambda x: 1 if x>=.5 else 0)
print(confusion_matrix(pred, y_test))

In [ ]:
(180+4918)/(180+4918+71772+302)

In [ ]:
# LGBM2

clf = lgb.LGBMClassifier(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.01)

fit_params={"early_stopping_rounds":1000, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': 'auto' # that's actually the default
           }

clf.fit(X_train, y_train, **fit_params)

In [ ]:
pred = clf.predict(X_test)
print(confusion_matrix(pred, y_test))

In [ ]:
# LGBM 3

fit_params = {
    "objective": "binary",
    "metric": "binary_error",
    "boosting": 'gbdt',
    "max_depth": -1,
    "num_leaves": 13,
    "learning_rate": 0.01,
    "bagging_freq": 5,
    "bagging_fraction": 0.4,
    "feature_fraction": 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    # "lambda_l1" : 5,
    # "lambda_l2" : 5,
    "bagging_seed": 314,
    "verbosity": 1,
    "seed": 314
}


evals_result = {}
lgb_clf = lgb.train(fit_params,
                            lgb_train,
                            100000,
                            valid_sets=[lgb_train, lgb_test],
                            early_stopping_rounds=3000,
                            verbose_eval=1000,
                            evals_result=evals_result
                            )

In [ ]:
pred = lgb_clf.predict(X_test)
pred = pd.Series(pred)
pred = pred.apply(lambda x: 1 if x>=.5 else 0)
print(confusion_matrix(pred, y_test))